In [114]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
llm=ChatOpenAI()

In [11]:
llm.invoke("What is langsmith?")

AIMessage(content='Langsmith is a surname of English origin. It is derived from the Old English words "lang," meaning long, and "smith," meaning someone who works with metal. This surname was likely given to someone who was a blacksmith or who worked with metal in some way.')

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

# Prompt template with HumanMessage and AIMessage
prompt = ChatPromptTemplate.from_messages([
    HumanMessage(content="What is langchain?"),
    AIMessage(content="Langchain is a platform for building and deploying language models."),
    HumanMessage(content="{input}")
])

In [19]:
chain = prompt | llm
chain.invoke({"input":"Tell me more about it."})

AIMessage(content='I\'m sorry, but I do not have any additional information on "langchain" as it appears to be a made-up term or a very niche topic with limited publicly available information. If you have more context or details about what you are referring to, I would be happy to try to provide more assistance.')

In [20]:
from langchain_core.output_parsers import StrOutputParser

# String output parser
output_parser = StrOutputParser()

In [21]:
# Chain with prompt, llm and output_parser
chain = prompt | llm | output_parser
chain.invoke({"input":"Tell me more about it."})

'Langchain is a decentralized blockchain-based platform that aims to revolutionize the language industry by providing a secure and transparent ecosystem for language learning, translation, and communication. It leverages blockchain technology to create a decentralized marketplace where users can buy and sell language-related services, such as translation, interpretation, and language tutoring, without the need for intermediaries. The platform also features a token economy that incentivizes users to contribute to the community and rewards them for their participation. Overall, Langchain aims to make language learning and communication more accessible, efficient, and secure for people around the world.'

In [22]:
# Streaming output
for chunk in chain.stream({"input":"Tell me more about it."}):
    print(chunk, end="", flush=True)

I'm sorry, but I don't have any specific information on "langchain" as it seems to be a relatively unknown term or concept. If you have any further context or details about what you are referring to, I would be happy to try to provide more information or assistance.

In [62]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()

In [63]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [82]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
documents = text_splitter.split_documents(docs)
vector = Chroma.from_documents(documents, embeddings)

In [94]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [95]:
from langchain.chains import create_retrieval_chain

# Retrieve k relevant documents
retriever = vector.as_retriever(search_kwargs={"k": 2})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [86]:
response = retrieval_chain.invoke({"input": "Tell me more about langchain?"})
print(response['answer'])

Langchain is a library available in both Python and JavaScript that provides documentation for the core APIs of LangChain. It appears to be a tool for language processing and manipulation.


In [110]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [105]:
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes")
]
retriever_chain.invoke({"chat_history": chat_history, "input": "Tell me step by step."})

[Document(page_content='Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com', 'title': 'LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com', 'title': 'LangSmith | 🦜️🛠️ LangSmith'})]

In [111]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the following question based only on the provided context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)
response = retrieval_chain.invoke({"chat_history": chat_history, "input": "Tell me step by step."})
print(response['answer'])

I'm sorry for any confusion, but based on the provided context, I can only see the name "LangSmith | 🦜️🛠️ LangSmith." I do not have any specific information about the services or capabilities of LangSmith. It would be best to reach out directly to LangSmith to inquire about their ability to help test your LLM applications and to get detailed step-by-step instructions on how to proceed.


In [112]:
from langchain.tools.retriever import create_retriever_tool
retreiver_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information relevant to Langsmith. For any questions about LangSmith, you must use this tool!")

In [115]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [122]:
tools = [retreiver_tool, search]

from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [119]:
agent_executor.invoke({"input": "What can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'What can LangSmith help with testing?'}`


LangSmith | 🦜️🛠️ LangSmith

LangSmith | 🦜️🛠️ LangSmithLangSmith can help with testing. If you have specific questions or need more information about how LangSmith can help with testing, feel free to ask!

> Finished chain.


{'input': 'What can langsmith help with testing?',
 'output': 'LangSmith can help with testing. If you have specific questions or need more information about how LangSmith can help with testing, feel free to ask!'}

In [121]:
agent_executor.invoke({"input": "what is the weather like today in singapore ?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Singapore today'}`


[{'url': 'https://www.weatherapi.com/', 'content': "Weather in Singapore is {'location': {'name': 'Singapore', 'region': '', 'country': 'Singapore', 'lat': 1.29, 'lon': 103.86, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1708785995, 'localtime': '2024-02-24 22:46'}, 'current': {'last_updated_epoch': 1708785900, 'last_updated': '2024-02-24 22:45', 'temp_c': 27.0, 'temp_f': 80.6, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 30, 'wind_dir': 'NNE', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 79, 'cloud': 75, 'feelslike_c': 30.4, 'feelslike_f': 86.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 19.8, 'gust_kph': 31.9}}"}, {'url': 'https://world-weather.info/forecast/singap

{'input': 'what is the weather like today in singapore ?',
 'output': 'The weather in Singapore today is partly cloudy with a temperature of 27.0°C (80.6°F). The wind is coming from the north-northeast at 15.1 km/h (9.4 mph), and the humidity is at 79%.'}